## In NoteBook

说明:

+ 在NoteBook中运行
+ 了解原作者思路
+ 了解改编后，策略是否是否能够正常运行
+ 适用于python3.7
+ 测试策略优化部分

In [1]:
%load_ext zipline

到目前为止，我们已经在研究中创建并运行了一条管道。现在我们转到IDE。首先，我们创建一个骨架算法，导入`Pipeline`，并为其添加一个`make_pipeline`函数，以创建一个空管道。

In [2]:
from zipline.pipeline import Pipeline

def initialize(context):
    my_pipe = make_pipeline()

def make_pipeline():
    return Pipeline()

## Attaching a Pipeline

回想一下，在研究中，我们运行了`make_pipeline`来创建管道对象的实例，并使用`run_pipeline`在指定的日期范围内运行管道。我们不能在算法中安全地做到这一点，我们必须以某种方式允许模拟为我们运行我们的管道。为了让仿真运行我们的管道，我们必须将管道与attach_pipeline连接起来。

`attach_pipeline`函数需要两个参数：对我们的管道对象的引用，以及可以任意决定的管道的字符串名称。让我们导入`attach_pipeline`并将我们的空管道连接到我们的骨架示例中。

In [3]:
from zipline.pipeline import Pipeline
from zipline.api import attach_pipeline

def initialize(context):
    my_pipe = make_pipeline()
    attach_pipeline(my_pipe, 'my_pipeline')

def make_pipeline():
    return Pipeline()

现在管道已连接，它将模拟每一天运行一次。如果算法在2016年6月6日星期一至2016年6月10日星期五进行回测或实时交易，我们的管道将在每周的每一天运行一次（总共5次）。附加管道将每天产生一个新的数据框输出。研究中的日输出与`run_pipeline`的输出类似。但是，我们算法中的输出数据框不包括索引中的日期，因为当前模拟日期暗示为我们管道计算的日期。

## Pipeline Output

管道的输出可以在`before_trading_start`中使用`pipeline_output`进行检索。`pipeline_output`需要附加管道的名称作为参数，并返回模拟中当前日期的输出数据框。让我们导入`pipeline_output`并修改我们的骨架示例，以便每天在上下文中存储我们的管道输出。

In [4]:
#%%zipline --start 2016-1-1 --end 2018-1-1
from zipline.pipeline import Pipeline
from zipline.api import attach_pipeline, pipeline_output

def initialize(context):
    my_pipe = make_pipeline()
    attach_pipeline(my_pipe, 'my_pipeline')

def make_pipeline():
    return Pipeline()

def before_trading_start(context, data):
    # Store our pipeline output DataFrame in context.
    #output = pipeline_output('my_pipeline')
    context.output = pipeline_output('my_pipeline')

我们的骨架示例现在每天生成3000行(期间有效证券数量)和默认指标数量列的empy数据框。 输出数据框将如下所示（请注意，该索引不再像研究中的MultiIndex一样）：

## Using Our Pipeline From Research

为了在我们的算法中包含我们在前一课中创建的管道，我们可以简单地将我们在研究中编写的`make_pipeline`函数以及所需的import语句复制到我们的算法中。以下内容将运行我们的管道，并在每天的上下文中存储150行和2列（多和空）的输出数据框。

In [5]:
%%zipline --start 2016-1-1 --end 2018-1-1
from zipline.pipeline.domain import CN_EQUITIES
from zipline.pipeline.builtin import QTradableStocks
from zipline.pipeline.factors import SimpleMovingAverage
from zipline.pipeline.data import CNEquityPricing
from zipline.api import attach_pipeline, pipeline_output
from zipline.pipeline import Pipeline


def initialize(context):
    my_pipe = make_pipeline()
    attach_pipeline(my_pipe, 'my_pipeline')


def make_pipeline():
    """
    Create our pipeline.
    """

    # Base universe set to the QTradableStocksUS.
    base_universe = QTradableStocks()

    # 10-day close price average.
    mean_10 = SimpleMovingAverage(
        inputs=[CNEquityPricing.close],
        window_length=10,
        mask=base_universe
    )

    # 30-day close price average.
    mean_30 = SimpleMovingAverage(
        inputs=[CNEquityPricing.close],
        window_length=30,
        mask=base_universe
    )

    percent_difference = (mean_10 - mean_30) / mean_30

    # Filter to select securities to short.
    shorts = percent_difference.top(75)

    # Filter to select securities to long.
    longs = percent_difference.bottom(75)

    # Filter for all securities that we want to trade.
    securities_to_trade = (shorts | longs)

    return Pipeline(
        columns={
            'longs': longs,
            'shorts': shorts
        },
        screen=(securities_to_trade),
        domain=CN_EQUITIES,
    )


def before_trading_start(context, data):
    # Store our pipeline output DataFrame in context
    output = pipeline_output('my_pipeline')

,period_open,period_close,long_value,returns,short_value,pnl,capital_used,long_exposure,orders,transactions,...,alpha,beta,sharpe,sortino,max_drawdown,max_leverage,excess_return,treasury_period_return,trading_days,period_label
2016-01-04 07:01:00+00:00,2016-01-04 01:31:00+00:00,2016-01-04 07:01:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,[],[],...,NaN,NaN,None,None,0.0,0.0,0.0,0.0,1,2016-01
2016-01-05 07:01:00+00:00,2016-01-05 01:31:00+00:00,2016-01-05 07:01:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,[],[],...,0.0,0.0,None,None,0.0,0.0,0.0,0.0,2,2016-01
2016-01-06 07:01:00+00:00,2016-01-06 01:31:00+00:00,2016-01-06 07:01:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,[],[],...,0.0,0.0,None,None,0.0,0.0,0.0,0.0,3,2016-01
2016-01-07 07:01:00+00:00,2016-01-07 01:31:00+00:00,2016-01-07 07:01:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,[],[],...,0.0,0.0,None,None,0.0,0.0,0.0,0.0,4,2016-01
2016-01-08 07:01:00+00:00,2016-01-08 01:31:00+00:00,2016-01-08 07:01:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,[],[],...,0.0,0.0,None,None,0.0,0.0,0.0,0.0,5,2016-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-25 07:01:00+00:00,2017-12-25 01:31:00+00:00,2017-12-25 07:01:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,[],[],...,0.0,0.0,None,None,0.0,0.0,0.0,0.0,484,2017-12
2017-12-26 07:01:00+00:00,2017-12-26 01:31:00+00:00,2017-12-26 07:01:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,[],[],...,0.0,0.0,None,None,0.0,0.0,0.0,0.0,485,2017-12
2017-12-27 07:01:00+00:00,2017-12-27 01:31:00+00:00,2017-12-27 07:01:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,[],[],...,0.0,0.0,None,None,0.0,0.0,0.0,0.0,486,2017-12
2017-12-28 07:01:00+00:00,2017-12-28 01:31:00+00:00,2017-12-28 07:01:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,[],[],...,0.0,0.0,None,None,0.0,0.0,0.0,0.0,487,2017-12


然后，我们可以定义一些函数来计算目标权重并根据我们的管道输出指定我们的多头和空头头寸的订单。让我们使用我们在入门教程中学到的一些基础知识来实现权重计算和下单。

最后，让我们把所有东西放在一起并记录一些变量。指定算法每周重新平衡。

In [6]:
%%zipline --start 2016-1-1 --end 2018-1-1

from zipline.api import order_optimal_portfolio, attach_pipeline, pipeline_output, schedule_function, date_rules, time_rules, record, order_percent
from zipline.pipeline import Pipeline
from zipline.pipeline.data import CNEquityPricing
from zipline.pipeline.factors import SimpleMovingAverage
from zipline.pipeline.builtin import QTradableStocksUS

import zipline.optimize as opt


def initialize(context):
    # Schedule our rebalance function to run at the start of
    # each week, when the market opens.
    schedule_function(my_rebalance, date_rules.week_start(),
                      time_rules.market_open())

    # Record variables at the end of each day.
    schedule_function(my_record_vars, date_rules.every_day(),
                      time_rules.market_close())

    # Create our pipeline and attach it to our algorithm.
    my_pipe = make_pipeline()
    attach_pipeline(my_pipe, 'my_pipeline')


def make_pipeline():
    """
    Create our pipeline.
    """

    # Base universe set to the QTradableStocksUS.
    base_universe = QTradableStocksUS()

    # 10-day close price average.
    mean_10 = SimpleMovingAverage(
        inputs=[CNEquityPricing.close], window_length=10, mask=base_universe)

    # 30-day close price average.
    mean_30 = SimpleMovingAverage(
        inputs=[CNEquityPricing.close], window_length=30, mask=base_universe)

    percent_difference = (mean_10 - mean_30) / mean_30

    # Filter to select securities to short.
    shorts = percent_difference.top(75)

    # Filter to select securities to long.
    longs = percent_difference.bottom(75)

    # Filter for all securities that we want to trade.
    securities_to_trade = (shorts | longs)

    return Pipeline(
        columns={
            'longs': longs,
            'shorts': shorts
        },
        screen=(securities_to_trade),
    )


def compute_target_weights(context, data):
    """
    Compute ordering weights.
    """

    # Initialize empty target weights dictionary.
    # This will map securities to their target weight.
    weights = {}

    # If there are securities in our longs and shorts lists,
    # compute even target weights for each security.
    if context.longs and context.shorts:
        long_weight = 0.5 / len(context.longs)
        short_weight = -0.5 / len(context.shorts)
    else:
        return weights

    # Exit positions in our portfolio if they are not
    # in our longs or shorts lists.
    for security in context.portfolio.positions:
        if security not in context.longs and security not in context.shorts and data.can_trade(
                security):
            weights[security] = 0

    for security in context.longs:
        weights[security] = long_weight

    for security in context.shorts:
        weights[security] = short_weight

    return weights


def before_trading_start(context, data):
    """
    Get pipeline results.
    """

    # Gets our pipeline output every day.
    pipe_results = pipeline_output('my_pipeline')

    # Go long in securities for which the 'longs' value is True,
    # and check if they can be traded.
    context.longs = []
    for sec in pipe_results[pipe_results['longs']].index.tolist():
        if data.can_trade(sec):
            context.longs.append(sec)

    # Go short in securities for which the 'shorts' value is True,
    # and check if they can be traded.
    context.shorts = []
    for sec in pipe_results[pipe_results['shorts']].index.tolist():
        if data.can_trade(sec):
            context.shorts.append(sec)


def my_rebalance(context, data):
    """
    Rebalance weekly.
    """
    # Calculate target weights to rebalance
    target_weights = compute_target_weights(context, data)
    for s, w in target_weights.items():
        order_percent(s, w)

    # If we have target weights, rebalance our portfolio
    if target_weights:
        order_optimal_portfolio(
            objective=opt.TargetWeights(target_weights),
            constraints=[],
        )


def my_record_vars(context, data):
    """
    Record variables at the end of each day.
    """

    longs = shorts = 0
    for _, position in context.portfolio.positions.items():
        if position.amount > 0:
            longs += 1
        elif position.amount < 0:
            shorts += 1

    # Record our variables.
    record(
        leverage=context.account.leverage,
        long_count=longs,
        short_count=shorts)

ImportError: cannot import name 'order_optimal_portfolio' from 'zipline.api' (c:\users\liudengfeng\repos\zipline\zipline\api.py)

**以上运行时间：2分9秒**

注意：当流水线在回测运行时，计算会分批执行以提高整体计算速度。由于计算是批量执行的，因此性能图表会显示为暂停。

祝贺您完成Pipeline教程！ 尝试在研究中设计一个管道，并在您自己的算法中使用它。